# Time Prediction Model using Multivariate Regression

## Load Data

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv('BPI2012Training.csv')
train_data.head()

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875
4,4294967296,173691,2011-10-01T08:08:58.256+02:00,5000,A_SUBMITTED,COMPLETE,01-10-2011 08:08:58.256


In [3]:
test_data = pd.read_csv('BPI2012Test.csv')
test_data.head()

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,44964012621824,206324,2012-02-03T17:17:11.047+01:00,2500,A_SUBMITTED,COMPLETE,03-02-2012 17:17:11.047
1,44964012621825,206324,2012-02-03T17:17:11.047+01:00,2500,A_PARTLYSUBMITTED,COMPLETE,03-02-2012 17:17:11.323
2,44964012621826,206324,2012-02-03T17:17:11.047+01:00,2500,A_DECLINED,COMPLETE,03-02-2012 17:17:42.964
3,44968307589120,206327,2012-02-03T17:23:41.949+01:00,6000,A_SUBMITTED,COMPLETE,03-02-2012 17:23:41.949
4,44968307589121,206327,2012-02-03T17:23:41.949+01:00,6000,A_PARTLYSUBMITTED,COMPLETE,03-02-2012 17:23:42.504


## Pre-processing 
### Functions

In [4]:
import datetime as dt

# Function that parses the incoming data set


def parseData(dataSet):
    # Parse time zone if there are any
    def convertToUnix(x):
        # If there is a timezone in the timestamp
        if 'T' in x:
            # Remove the T
            without_timezone = x[:10] + ' ' + x[11:-6]

            # Parse milliseconds if contained
            if '.' in x:
                wholesomeTime = dt.datetime.timestamp(
                    dt.datetime.strptime(without_timezone, "%Y-%m-%d %H:%M:%S.%f"))
            else:
                wholesomeTime = dt.datetime.timestamp(
                    dt.datetime.strptime(without_timezone, "%Y-%m-%d %H:%M:%S"))

        else:
            if '.' in x:
                wholesomeTime = dt.datetime.timestamp(
                    dt.datetime.strptime(x, "%d-%m-%Y %H:%M:%S.%f"))
            else:
                wholesomeTime = dt.datetime.timestamp(
                    dt.datetime.strptime(x, "%d-%m-%Y %H:%M:%S"))

        return wholesomeTime

    # Convert absolute event and reg timestamp into unix time
    dataSet['unix_abs_event_time'] = dataSet['event time:timestamp'].apply(
        lambda x: convertToUnix(x))
    dataSet['unix_reg_time'] = dataSet['case REG_DATE'].apply(
        lambda x: convertToUnix(x))

    # Time it takes for an event to occur from registeration
    dataSet['unix_rel_event_time'] = dataSet['unix_abs_event_time'] - \
        dataSet['unix_reg_time']

    # Group data set by case ID
    dataSet_grouped_by_case = dataSet.groupby(by=['case concept:name'])

    # Return data frame consisting out of the last event per case with column that 
    #indicates the number of events the case underwent appended
    dataSet_last_event_per_case = dataSet_grouped_by_case.nth([-1])
    dataSet_last_event_per_case['num_events'] = dataSet_grouped_by_case.count(
    ).iloc[:, 0]

    return (dataSet, dataSet_last_event_per_case)

In [5]:
def timeToNextEvent(dataSet):
    df_predicted_time_to_next_event = dataSet.copy().sort_values(
        by=['case concept:name', "eventID ", "unix_abs_event_time"])

    df_predicted_time_to_next_event['actual_time_to_next_event'] = df_predicted_time_to_next_event['unix_abs_event_time'].shift(
        -1) - df_predicted_time_to_next_event["unix_abs_event_time"]

    df_predicted_time_to_next_event['actual_time_to_next_event'] = np.where((df_predicted_time_to_next_event['case concept:name'] == df_predicted_time_to_next_event['case concept:name'].shift(
        -1)), df_predicted_time_to_next_event['actual_time_to_next_event'], np.nan)
    
    df = df_predicted_time_to_next_event[df_predicted_time_to_next_event['actual_time_to_next_event'].notna()]
    
    return (df)

In [6]:
def oneHotEncoding(dataSet, attr):
    one_hot = pd.get_dummies(dataSet[attr])
    df = dataSet.join(one_hot)
    
    return (df)

In [7]:
def eventStartHour(dataSet):
    dataSet['event time:timestamp'] = pd.to_datetime(dataSet['event time:timestamp'])
    dataSet['event_start_hour'] = dataSet['event time:timestamp'].dt.hour
    
    return(dataSet)

### Pre-processing Data

In [8]:
df_train = parseData(train_data)
df_train = timeToNextEvent(df_train[0])
df_train = oneHotEncoding(df_train, 'event concept:name')
df_train = eventStartHour(df_train)
df_train.head()

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp,unix_abs_event_time,unix_reg_time,unix_rel_event_time,...,O_SENT,O_SENT_BACK,W_Afhandelen leads,W_Beoordelen fraude,W_Completeren aanvraag,W_Nabellen incomplete dossiers,W_Nabellen offertes,W_Valideren aanvraag,W_Wijzigen contractgegevens,event_start_hour
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,2011-01-10 00:38:44.546,1.317422e+09,1.317422e+09,0.000,...,0,0,0,0,0,0,0,0,0,0
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,2011-01-10 00:38:44.880,1.317422e+09,1.317422e+09,0.334,...,0,0,0,0,0,0,0,0,0,0
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,2011-01-10 00:39:37.906,1.317422e+09,1.317422e+09,53.360,...,0,0,0,0,0,0,0,0,0,0
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,2011-01-10 00:39:38.875,1.317422e+09,1.317422e+09,54.329,...,0,0,0,0,1,0,0,0,0,0
89,4,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,START,2011-01-10 11:36:46.437,1.317462e+09,1.317422e+09,39481.891,...,0,0,0,0,1,0,0,0,0,11


In [9]:
df_test = parseData(test_data)
df_test = timeToNextEvent(df_test[0])
df_test = oneHotEncoding(df_test, 'event concept:name')
df_test = eventStartHour(df_test)
df_test.head()

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp,unix_abs_event_time,unix_reg_time,unix_rel_event_time,...,O_SELECTED,O_SENT,O_SENT_BACK,W_Afhandelen leads,W_Beoordelen fraude,W_Completeren aanvraag,W_Nabellen incomplete dossiers,W_Nabellen offertes,W_Valideren aanvraag,event_start_hour
0,44964012621824,206324,2012-02-03T17:17:11.047+01:00,2500,A_SUBMITTED,COMPLETE,2012-03-02 17:17:11.047,1.328286e+09,1.328286e+09,0.000,...,0,0,0,0,0,0,0,0,0,17
1,44964012621825,206324,2012-02-03T17:17:11.047+01:00,2500,A_PARTLYSUBMITTED,COMPLETE,2012-03-02 17:17:11.323,1.328286e+09,1.328286e+09,0.276,...,0,0,0,0,0,0,0,0,0,17
3,44968307589120,206327,2012-02-03T17:23:41.949+01:00,6000,A_SUBMITTED,COMPLETE,2012-03-02 17:23:41.949,1.328286e+09,1.328286e+09,0.000,...,0,0,0,0,0,0,0,0,0,17
4,44968307589121,206327,2012-02-03T17:23:41.949+01:00,6000,A_PARTLYSUBMITTED,COMPLETE,2012-03-02 17:23:42.504,1.328286e+09,1.328286e+09,0.555,...,0,0,0,0,0,0,0,0,0,17
5,44968307589122,206327,2012-02-03T17:23:41.949+01:00,6000,A_PREACCEPTED,COMPLETE,2012-03-02 17:24:23.379,1.328286e+09,1.328286e+09,41.430,...,0,0,0,0,0,0,0,0,0,17


## Regression Model 

In [10]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from math import sqrt

def timePrediction(df_train, df_test):
    x_train = df_train[['case AMOUNT_REQ', 'event_start_hour', 'A_SUBMITTED', 'A_PARTLYSUBMITTED', 'A_PREACCEPTED',
       'W_Completeren aanvraag', 'A_DECLINED', 'W_Afhandelen leads',
       'A_ACCEPTED', 'O_SELECTED', 'A_FINALIZED', 'O_CREATED', 'O_SENT',
       'W_Nabellen offertes', 'O_CANCELLED', 'A_CANCELLED',
       'W_Beoordelen fraude', 'O_SENT_BACK', 'W_Valideren aanvraag',
       'W_Nabellen incomplete dossiers', 'O_ACCEPTED', 'A_APPROVED',
       'A_ACTIVATED', 'A_REGISTERED', 'O_DECLINED']].copy() 
    y_train = df_train[['actual_time_to_next_event']].copy()
    
    model = LinearRegression().fit(x_train, y_train)
    R2 = model.score(x_train, y_train)
    
    x_test = df_test[['case AMOUNT_REQ', 'event_start_hour', 'A_SUBMITTED', 'A_PARTLYSUBMITTED', 'A_PREACCEPTED',
       'W_Completeren aanvraag', 'A_DECLINED', 'W_Afhandelen leads',
       'A_ACCEPTED', 'O_SELECTED', 'A_FINALIZED', 'O_CREATED', 'O_SENT',
       'W_Nabellen offertes', 'O_CANCELLED', 'A_CANCELLED',
       'W_Beoordelen fraude', 'O_SENT_BACK', 'W_Valideren aanvraag',
       'W_Nabellen incomplete dossiers', 'O_ACCEPTED', 'A_APPROVED',
       'A_ACTIVATED', 'A_REGISTERED', 'O_DECLINED']].copy()
    y_test = df_test[['actual_time_to_next_event']].copy()
    
    time_pred = model.predict(x_test)
    y_test['predicted_time'] = time_pred
    
    RMSE = sqrt(metrics.mean_squared_error(y_test['actual_time_to_next_event'], y_test['predicted_time']))
    
    return(y_test, R2, RMSE)

### Predicting the time taken from the start of an event to the start of the next event

In [11]:
pred_model = timePrediction(df_train, df_test)

In [12]:
#dataframe showing actual time and predicted time
pred_model[0].head()

,actual_time_to_next_event,predicted_time
0,0.276,2308.060453
1,31.641,2342.484102
3,0.555,2241.114387
4,40.875,2275.538036
5,0.673,2837.067595


In [13]:
#Model quality assessment using R-squared
pred_model[1]

0.08084613327911583

In [14]:
#accuracy Evaluation using Mean Squared Error 
pred_model[2]

125085.81646692034

### Evaluation

RMSE of prediction model: 125085.81646692034

MSE of prediction model: 15646461481.196081